In [1]:
from pathlib import Path
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as tvtfs
from torch.utils.data import Dataset, DataLoader
import datasets as hfds
from transformers import ViTForImageClassification, ViTImageProcessor, get_scheduler
from tqdm import tqdm

In [2]:
# def load_model(model_name_or_path: str = "google/vit-base-patch16-224"):
#     return ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=1000)

def load_model():
    return torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

In [3]:
class ImagenetDataset(Dataset):
    """ Imagenet dataset with random masking """
    def __init__(self, split: str, rand_masks: bool = False):
        self.split = split
        self.ds = hfds.load_dataset("ILSVRC/imagenet-1k", split=split, cache_dir="/shared_data0/antonxue/")
        self.processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
        self.rand_masks = rand_masks

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        image = item["image"]
        image = self.processor(
            image if image.mode == "RGB" else image.convert("RGB"),
            return_tensors = "pt",
        )["pixel_values"].view(3,224,224)
        # Each 16x16 patch (14x14 grid) has a random chance of turning off, randomly.
        if self.rand_masks:
            small_mask = torch.rand(1,1,14,14) < torch.rand(())
            large_mask = F.interpolate(small_mask.float(), scale_factor=(16,16))
            image = image.view(3,224,224) * large_mask.view(1,224,224)
        return {"image": image, "label": item["label"]}

In [4]:
len(ImagenetDataset(split="train"))

Using the latest cached version of the dataset since ILSVRC/imagenet-1k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /shared_data0/antonxue/ILSVRC___imagenet-1k/default/1.0.0/07900defe1ccf3404ea7e5e876a64ca41192f6c07406044771544ef1505831e8 (last modified on Sat Jan 25 21:31:50 2025).


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

1281167

In [5]:
len(ImagenetDataset(split="test"))

Using the latest cached version of the dataset since ILSVRC/imagenet-1k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /shared_data0/antonxue/ILSVRC___imagenet-1k/default/1.0.0/07900defe1ccf3404ea7e5e876a64ca41192f6c07406044771544ef1505831e8 (last modified on Sat Jan 25 21:31:50 2025).


Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

100000

In [6]:
def train_one_epoch(model, dataloader, optimizer, lr_scheduler):
    model.train()
    device = next(model.parameters()).device
    all_losses, all_accs = [], []
    pbar = tqdm(dataloader)
    for batch in pbar:
        image, label = batch["image"].to(device), batch["label"].to(device)
        out = model(image)
        if hasattr(out, "logits"):
            logits = out.logits
        elif isinstance(out, dict) and "logits" in out.keys():
            logits = out["logits"]
        else:
            logits = out
        loss = F.cross_entropy(logits, label)
        loss.backward(); optimizer.step(); optimizer.zero_grad(); lr_scheduler.step()

        all_losses.append(loss.detach().item())
        all_accs.append((logits.argmax(dim=-1) == label).float().mean().detach().item())
        pbar.set_description(
            f"lr {lr_scheduler.get_last_lr()[0]:.3e}, "
            + f"loss {torch.tensor(all_losses[-16:]).mean():.3e}, "
            + f"acc {torch.tensor(all_accs[-16:]).float().mean():.3e}"
        )
    return {"state_dict": model.state_dict(), "train_losses": all_losses, "train_accs": all_accs}

In [7]:
@torch.no_grad()
def valid_one_epoch(model, dataloader):
    model.eval()
    device = next(model.parameters()).device
    all_losses, all_accs = [], []
    pbar = tqdm(dataloader)
    for batch in pbar:
        image, label = batch["image"].to(device), batch["label"].to(device)
        logits = model(image).logits
        loss = F.cross_entropy(logits, label)

        all_losses.append(loss.detach().item())
        all_accs.append((logits.argmax(dim=-1) == label).float().mean().detach().item())
        pbar.set_description(
            f"loss {torch.tensor(all_losses[-16:]).mean():.3e}, "
            + f"acc {torch.tensor(all_accs[-16:]).float().mean():.3e}"
        )
    return {"valid_losses": all_losses, "valid_accs": all_accs}

In [8]:
def run_finetuning_epochs(
    num_epochs: int = 5,
    bsz: int = 1024,
    lr: float = 1e-4,
    device: str = "cuda",
    saveto_dir: str = "../_saved_models",
):
    # Make directory if doesn't exist
    Path(saveto_dir).mkdir(parents=True, exist_ok=True)

    # Initialize dataloaders
    train_dataloader = DataLoader(ImagenetDataset("train", rand_masks=True), batch_size=bsz, num_workers=24, shuffle=True)
    valid_dataloader = DataLoader(ImagenetDataset("validation", rand_masks=True), batch_size=bsz, num_workers=24, shuffle=True)

    # Set up the model, optimizer, and learning rate scheduler
    model = load_model()
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    num_train_steps = len(train_dataloader) * num_epochs
    lr_scheduler = get_scheduler(
        name = "linear",
        optimizer = optimizer,
        num_warmup_steps = int(0.1 * num_train_steps),
        num_training_steps = num_train_steps
    )

    # Fine-tuning loop
    for epoch in range(1, num_epochs+1):
        # saveto = f"{saveto_dir}/google_vit_patch16_img224_bsz{bsz}_lr{lr:.4f}_epoch{epoch}.pt"
        saveto = f"{saveto_dir}/resnet18_patch16_img224_bsz{bsz}_lr{lr:.4f}_epoch{epoch}.pt"
        print("Training", saveto)
        train_ret = train_one_epoch(model, train_dataloader, optimizer, lr_scheduler)
        valid_ret = valid_one_epoch(model, valid_dataloader)
        train_ret["valid_losses"] = valid_ret["valid_losses"]
        train_ret["valid_accs"] = valid_ret["valid_accs"]
        torch.save(train_ret, saveto)

    return train_ret

In [9]:
run_finetuning_epochs(num_epochs=1)

Using the latest cached version of the dataset since ILSVRC/imagenet-1k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /shared_data0/antonxue/ILSVRC___imagenet-1k/default/1.0.0/07900defe1ccf3404ea7e5e876a64ca41192f6c07406044771544ef1505831e8 (last modified on Sat Jan 25 21:31:50 2025).


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Using the latest cached version of the dataset since ILSVRC/imagenet-1k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /shared_data0/antonxue/ILSVRC___imagenet-1k/default/1.0.0/07900defe1ccf3404ea7e5e876a64ca41192f6c07406044771544ef1505831e8 (last modified on Sat Jan 25 21:31:50 2025).


Training ../_saved_models/resnet18_patch16_img224_bsz1024_lr0.0001_epoch1.pt


lr 1.920e-05, loss 4.305e+00, acc 2.570e-01:   2%|▋                                  | 24/1252 [07:49<16:43,  1.22it/s]/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
lr 9.600e-05, loss 2.549e+00, acc 4.628e-01:  10%|███▎                              | 120/1252 [32:38<15:14,  1.24it/s]/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
lr 8.554e-05, loss 2.173e+00, acc 5.201e-01:  23%|███████▎                        | 288/1252 [1:06:48<26:55,  1.68s/it]/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:648: UserWarning: Metadata Warning, tag 274 had too many entries: 4, e

KeyboardInterrupt: 